In [0]:
try:
  # %tensorflow_version only exists in Colab.
  #%tensorflow_version 2.x
  %tensorflow_version 1.x  
except Exception:
  pass

# Load the TensorBoard notebook extension
#%load_ext tensorboard

In [0]:
import tensorflow as tf
import datetime

In [0]:
from os import makedirs
from os.path import exists, join
import numpy as np

import keras
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, concatenate, Input
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [0]:
log_dir= "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

if not exists(log_dir):
    makedirs(log_dir)

# set parameters:
max_features = 5000 # Put it up to 10k?
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 125 # play with this too
kernel_size = 3
hidden_dims = 125
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

# save class labels to disk to color data points in TensorBoard accordingly
with open(join(log_dir, 'metadata.tsv'), 'w') as f:
    np.savetxt(f, y_test)

tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1, 
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=x_test)

Loading data...
17465344/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [0]:
var_input = Input(shape=(400,))

emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)

kernel3 = (Conv1D(filters,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=1))(emb_layer)
kernel3 = GlobalMaxPooling1D()(kernel3)
kernel4 = (Conv1D(filters,
                 kernel_size=4,
                 padding='valid',
                 activation='relu',
                 strides=1))(emb_layer)
kernel4 = GlobalMaxPooling1D()(kernel4)
kernel5 = (Conv1D(filters,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))(emb_layer)
kernel5 = GlobalMaxPooling1D()(kernel5)

output = concatenate([kernel3, kernel4, kernel5])
output = Dense(1, activation=None, name='features')(output)
act_output = Activation('sigmoid')(output)
model = Model(inputs=var_input, outputs=act_output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size, 
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))




Train on 25000 samples, validate on 25000 samples
Epoch 1/2





25000/25000 [==============================] - 22s 868us/step - loss: 0.3929 - acc: 0.8176 - val_loss: 0.2707 - val_acc: 0.8885
Epoch 2/2
25000/25000 [==============================] - 15s 585us/step - loss: 0.2072 - acc: 0.9182 - val_loss: 0.2681 - val_acc: 0.8894


In [0]:
# Add dropout to previous model
var_input = Input(shape=(400,))

emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)

kernel3 = (Conv1D(filters,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=1))(emb_layer)
kernel3 = GlobalMaxPooling1D()(kernel3)
kernel4 = (Conv1D(filters,
                 kernel_size=4,
                 padding='valid',
                 activation='relu',
                 strides=1))(emb_layer)
kernel4 = GlobalMaxPooling1D()(kernel4)
kernel5 = (Conv1D(filters,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))(emb_layer)
kernel5 = GlobalMaxPooling1D()(kernel5)

output = concatenate([kernel3, kernel4, kernel5])
output = Dense(1, activation=None, name='features')(output)
dropout = Dropout(rate=0.2, seed=42)(output) # TODO tweak
act_output = Activation('sigmoid')(dropout)
model = Model(inputs=var_input, outputs=act_output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size, 
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 7s 273us/step - loss: 0.4570 - acc: 0.7519 - val_loss: 0.2872 - val_acc: 0.8845
Epoch 2/2
25000/25000 [==============================] - 6s 252us/step - loss: 0.3155 - acc: 0.8304 - val_loss: 0.2578 - val_acc: 0.8940


Adding 0.2 rate dropout decreases accuracy by almost 10 points ~0.91 to ~0.83. We expect something different using SELUs / ELUs??? 
Accuracy reported by paper "Effectiveness of SNNs" is very poor (78 ~ 82% )

In [0]:
# Add dropout to previous model
var_input = Input(shape=(400,))

emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)
initial_dropout = Dropout(rate=0.1, seed=42)(emb_layer)
kernel3 = (Conv1D(filters,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=1))(initial_dropout)
kernel3 = GlobalMaxPooling1D()(kernel3)
kernel4 = (Conv1D(filters,
                 kernel_size=4,
                 padding='valid',
                 activation='relu',
                 strides=1))(initial_dropout)
kernel4 = GlobalMaxPooling1D()(kernel4)
kernel5 = (Conv1D(filters,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))(initial_dropout)
kernel5 = GlobalMaxPooling1D()(kernel5)

output = concatenate([kernel3, kernel4, kernel5])
output = Dense(1, activation=None, name='features')(output)
#dropout = Dropout(rate=0.1, seed=42)(output) # TODO tweak
act_output = Activation('sigmoid')(output)
model = Model(inputs=var_input, outputs=act_output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size, 
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 25000 samples, validate on 25000 samples
Epoch 1/2





25000/25000 [==============================] - 13s 528us/step - loss: 0.4045 - acc: 0.8100 - val_loss: 0.2758 - val_acc: 0.8840
Epoch 2/2
25000/25000 [==============================] - 6s 252us/step - loss: 0.2260 - acc: 0.9110 - val_loss: 0.2569 - val_acc: 0.8945


Seems like adding dropout in the embedded layer does not improve accuracy. More tests needed?

---



In [0]:
var_input = Input(shape=(400,))

emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)

kernel3 = (Conv1D(filters,
                 kernel_size=3,
                 padding='valid',
                 activation='selu',
                 strides=1))(emb_layer)
kernel3 = GlobalMaxPooling1D()(kernel3)
kernel4 = (Conv1D(filters,
                 kernel_size=4,
                 padding='valid',
                 activation='selu',
                 strides=1))(emb_layer)
kernel4 = GlobalMaxPooling1D()(kernel4)
kernel5 = (Conv1D(filters,
                 kernel_size=5,
                 padding='valid',
                 activation='selu',
                 strides=1))(emb_layer)
kernel5 = GlobalMaxPooling1D()(kernel5)

output = concatenate([kernel3, kernel4, kernel5])
output = Dense(1, activation=None, name='features')(output)
act_output = Activation('sigmoid')(output)
model = Model(inputs=var_input, outputs=act_output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size, 
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 8s 314us/step - loss: 0.3911 - acc: 0.8170 - val_loss: 0.2654 - val_acc: 0.8890
Epoch 2/2
25000/25000 [==============================] - 7s 292us/step - loss: 0.2093 - acc: 0.9175 - val_loss: 0.2640 - val_acc: 0.8898


As expected, SELU activation in such a shallow network does not have a notable impact on accuracy.



In [0]:
#%tensorboard --logdir logs/fit